In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv("GPT-wiki-intro.csv")

In [3]:
data = data[:100000]

In [4]:
data

,id,url,title,wiki_intro,generated_intro,title_len,wiki_intro_len,generated_intro_len,prompt,generated_text,prompt_tokens,generated_text_tokens
0,63064638,https://en.wikipedia.org/wiki/Sexhow%20railway...,Sexhow railway station,Sexhow railway station was a railway station b...,Sexhow railway station was a railway station l...,3,174,78,200 word wikipedia style introduction on 'Sexh...,"located in the town of Sexhow, on the Cumbria...",25,88
1,279621,https://en.wikipedia.org/wiki/Eti%C3%A4inen,Etiäinen,"In Finnish folklore, all places and things, an...","In Finnish folklore, all places and things, an...",1,187,80,200 word wikipedia style introduction on 'Etiä...,"animate or inanimate, have a spirit or ""etiäi...",26,101
2,287229,https://en.wikipedia.org/wiki/Inverse%20functi...,Inverse function theorem,"In mathematics, specifically differential calc...","In mathematics, specifically differential calc...",3,170,59,200 word wikipedia style introduction on 'Inve...,function theorem states that for every real-v...,26,65
3,26712375,https://en.wikipedia.org/wiki/Stepping%20on%20...,Stepping on Roses,is a Japanese shōjo manga series written and i...,is a Japanese shōjo manga series written and i...,3,335,121,200 word wikipedia style introduction on 'Step...,and illustrated by Maki Fujii. The series fol...,26,150
4,38894426,https://en.wikipedia.org/wiki/Rob%20Bradley,Rob Bradley,"Robert Milner ""Rob"" Bradley, Jr. (born August ...","Robert Milner ""Rob"" Bradley, Jr. (born August ...",2,170,136,200 word wikipedia style introduction on 'Rob ...,"29, 1973) is an American former professional ...",28,162
...,...,...,...,...,...,...,...,...,...,...,...,...
99995,38050027,https://en.wikipedia.org/wiki/Hero%20Squad,Hero Squad,Hero Squad is a children's television programm...,Hero Squad is a children's television programm...,2,183,124,200 word wikipedia style introduction on 'Hero...,that airs on Nickelodeon in the United States...,23,153
99996,535861,https://en.wikipedia.org/wiki/The%20Regulators...,The Regulators (novel),The Regulators is a novel by American author S...,The Regulators is a novel by American author S...,3,159,86,200 word wikipedia style introduction on 'The ...,author Stephen King. It is set in the fiction...,27,98
99997,42257728,https://en.wikipedia.org/wiki/Caledonian%20Estate,Caledonian Estate,"The Caledonian Estate is a Grade II listed, ea...",The Caledonian Estate is a Grade II listed bui...,2,151,58,200 word wikipedia style introduction on 'Cale...,listed building located in the Scottish city ...,26,65
99998,19551524,https://en.wikipedia.org/wiki/Course%20%28orie...,Course (orienteering),An orienteering course is composed of a start ...,An orienteering course is composed of a series...,2,183,205,200 word wikipedia style introduction on 'Cour...,"series of checkpoints, or observation points,...",25,245


In [5]:
intro = []
target = []

In [6]:
for i in data["wiki_intro"]:
    intro.append(i)
    target.append(0)
for j in data["generated_intro"]:
    intro.append(j)
    target.append(1)

In [7]:
intros = np.array(intro)
targets = np.array(target)

In [8]:
intros.shape

(200000,)

In [9]:
from sklearn.model_selection import train_test_split,  GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn import neighbors
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
import time
from sklearn.ensemble import VotingClassifier
from sklearn.feature_extraction.text import CountVectorizer


In [10]:
X_train, X_test, y_train, y_test = train_test_split(intros, targets, test_size=0.2, random_state=42)

In [11]:
X_train.shape

(160000,)

In [12]:
# Créer un pipeline de préparation de données
vectorizer = CountVectorizer()  # Utiliser la représentation Bag-of-Words
X_train_preprocessed = vectorizer.fit_transform(X_train)
X_test_preprocessed = vectorizer.transform(X_test)

In [13]:
X_train_preprocessed

<160000x407976 sparse matrix of type '<class 'numpy.int64'>'
	with 15133953 stored elements in Compressed Sparse Row format>

In [14]:
X_test_preprocessed

<40000x407976 sparse matrix of type '<class 'numpy.int64'>'
	with 3713740 stored elements in Compressed Sparse Row format>

In [15]:
import joblib
joblib.dump(vectorizer, 'vectorizer')

['vectorizer']

In [16]:
random_forest = RandomForestClassifier(n_estimators=100)
t1 = time.time()
random_forest.fit(X_train_preprocessed, y_train)
t2 = time.time()

In [17]:
t2-t1

1396.2612698078156

In [18]:
# Prédictions sur l'ensemble de test
y_pred = random_forest.predict(X_test_preprocessed)

# Évaluation des performances
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9062
